In [3]:
import cv2

"""
Author: Valentina Matos (Johns Hopkins - Wirtz/Kiemen Lab)
Date: May 29, 2024
"""

import os
import tensorflow as tf
import numpy as np
from glob import glob
from PIL import Image
import matplotlib.pyplot as plt
import pickle
import time
from scipy.ndimage import binary_fill_holes
from Semanticseg import semantic_seg
from make_overlay import make_overlay
Image.MAX_IMAGE_PIXELS = None



In [4]:
#Inputs:
pthim = r'C:\Users\Valentina\OneDrive - Johns Hopkins\Desktop\test png\PNG'
pthDL = r'\\10.99.68.52\Kiemendata\Valentina Matos\coda to python\test model\04_19_2024'
color_overlay_HE=False #set to true in the real code
color_mask=False


In [5]:
# Load the model weights and other relevant data
with open(os.path.join(pthDL, 'net.pkl'), 'rb') as f:
    data = pickle.load(f)
    model = data['model']
    classNames = data['classNames']
    sxy = data['sxy']
    nblack = data['nblack']
    nwhite = data['nwhite']
    cmap = data['cmap']
    nm = data['nm']
sxy = 1024 #need to update

In [10]:
outpth = os.path.join(pthim, 'classification_'+ nm)
os.makedirs(outpth, exist_ok=True)

In [11]:
b=100
imlist = sorted(glob(os.path.join(pthim, '*.png')))
# If no PNGs found, search for JPG files
if not imlist:
  jpg_files = glob(os.path.join(pthim, "*.jpg"))
  if jpg_files:
    imlist.extend(jpg_files)  # Add full paths of JPGs to list
if not imlist:
  print("No PNG or JPG image files found in", pthim)
print('   ')

In [15]:
classification_st = time.time()
save_dirHE = r'C:\Users\Valentina\OneDrive - Johns Hopkins\Desktop\test png\PNG\tilesHE'
save_dirmask = r'C:\Users\Valentina\OneDrive - Johns Hopkins\Desktop\test png\PNG\tilesmask'
os.makedirs(save_dirHE, exist_ok=True)
os.makedirs(save_dirmask, exist_ok=True)

for i, img_path in enumerate(imlist):
    img_name = os.path.basename(img_path)
    print(f'  Starting classification of image {i+1} of {len(imlist)}: {img_name}')
    if os.path.isfile(os.path.join(outpth, img_name[:-4] + ".png")):
        print(f'  Image {img_name} already classified by this model')
        continue
    im = Image.open(os.path.join(pthim, img_name))
    im_array = np.array(im)  # Convert to NumPy array for slicing
    try:
        try:
            TA = Image.open(os.path.join(outpth, 'TA', img_name[:-4] + ".png"))
        except:
            TA = Image.open(os.path.join(outpth, 'TA', img_name[:-4] + ".tif"))
        TA = binary_fill_holes(TA)
    except:
        TA = np.array(im.convert('L')) < 220
        TA = binary_fill_holes(TA.astype(bool))


    imclassify = np.zeros(TA.shape, dtype=np.uint8)
    sz = np.array(im).shape
    
    count = 1
    for s1 in range(0, sxy - b * 2, sz[0] - sxy):
        for s2 in range(0, sxy - b * 2, sz[1] - sxy):
            tileHE = im_array[s1:s1 + sxy, s2:s2 + sxy, :]
            tileHE_image = Image.fromarray(tileHE)
            save_path_HE  = os.path.join(save_dirHE, f'{img_name}_tileHE_{count}.png')  # You can change the filename as needed
            tileHE_image.save(save_path_HE )
            
            
            tileTA = TA[s1:s1 + sxy, s2:s2 + sxy]

            if np.sum(tileTA) < 100:
                tileclassify = np.zeros(TA.shape)
            else:
                tileclassify = semantic_seg(tileHE, image_size=1024, model=model)
            
            tileclassify = tileclassify[b:-b, b:-b]
            
            # Convert to uint8 before saving
            tileclassify_uint8 = (tileclassify * 10).astype(np.uint8)
            tilemask_image = Image.fromarray(tileclassify_uint8)
            save_path_mask = os.path.join(save_dirmask, f'{img_name}_tilemask_{count}.png')
            tilemask_image.save(save_path_mask)
            
            imclassify[s1 + b:s1 + sxy - b, s2 + b:s2 + sxy - b] = tileclassify
            print(count)
            count += 1

    # Remove padding
    im_array = im_array[sxy + b:-sxy - b, sxy + b:-sxy - b, :]
    imclassify = imclassify[sxy + b:-sxy - b, sxy + b:-sxy - b]

    elapsed_time = round(time.time() - classification_st)
    print(f'{i+1} of {len(imlist)} {elapsed_time}')

    imclassify[np.logical_or(imclassify == nblack, imclassify == 0)] = nwhite
    imclassify = Image.fromarray(imclassify)  # Convert NumPy array to PIL Image
    imclassify.save(os.path.join(outpth, img_name[:-3] + 'png'))  # Save as PNG

    # Make color image overlay on H&E
    if color_overlay_HE:
        save_path = os.path.join(outpth, 'check_classification')
        _ = make_overlay(img_path, imclassify, image_size=sxy, colormap=cmap, save_path=save_path)

    if color_mask:
        outpthcolor = os.path.join(outpth, 'color')
        os.makedirs(outpthcolor, exist_ok=True)
        red_channel = cmap[:, 0]
        green_channel = cmap[:, 1]
        blue_channel = cmap[:, 2]

        imcolor = np.dstack((red_channel[imclassify], green_channel[imclassify], blue_channel[imclassify])).astype(np.uint8)

        save_file_path = os.path.join(outpthcolor, os.path.basename(img_path))
        cv2.imwrite(save_file_path, imcolor)

    # Display the first image in the series
    if i == 0 and cmap is not None:
        red_channel = cmap[:, 0]
        green_channel = cmap[:, 1]
        blue_channel = cmap[:, 2]

        imcolor = np.dstack((red_channel[imclassify], green_channel[imclassify], blue_channel[imclassify])).astype(np.uint8)

        fig, axs = plt.subplots(1, 2)
        axs[0].imshow(im)
        axs[1].imshow(imcolor)

        for ax in axs:
            ax.axis('off')

        plt.subplots_adjust(wspace=0, hspace=0)
        plt.show()
        plt.pause(0.2)


  Starting classification of image 1 of 2: SG_013_0061.png


KeyboardInterrupt: 

In [3]:
import os
import pickle
pthDL = r'\\10.99.68.52\Kiemendata\Valentina Matos\coda to python\test model\04_19_2024'
with open(os.path.join(pthDL, 'net.pkl'), 'rb') as f:
    data = pickle.load(f)
for key, value in data.items():
    print(f"Key: {key}, Value: {value}")

Key: pthim, Value: \\10.99.68.52\Kiemendata\Valentina Matos\coda to python\test model\5x
Key: pthDL, Value: \\10.99.68.52\Kiemendata\Valentina Matos\coda to python\test model\04_19_2024
Key: WS, Value: [[2, 0, 0, 1, 0, 0, 2, 0, 2, 2, 2, 0, 0], [7, 6], [1, 2, 3, 4, 5, 6, 7, 8, 7, 9, 10, 8, 11], [6, 5, 4, 11, 1, 2, 3, 8, 10, 12, 13, 7, 9], []]
Key: nm, Value: 04_19_2024
Key: umpix, Value: 2
Key: cmap, Value: [[121 248 252]
 [  0   0 255]
 [ 80 237  80]
 [255 255   0]
 [149  35 184]
 [255 194 245]
 [255 255 255]
 [255   0   0]
 [ 73 120 111]
 [  0   0   0]
 [240 159  10]]
Key: sxy, Value: 1000
Key: classNames, Value: ['islet', 'epithelium', 'vasculature', 'fat', 'acini', 'stroma', 'nontissue', 'PanIN', 'nerves', 'immune', 'PDAC', 'black']
Key: ntrain, Value: 15
Key: nblack, Value: 12
Key: nwhite, Value: 7
Key: nvalidate, Value: 3
Key: model, Value: <Functional name=functional_1, built=True>
Key: history, Value: {'accuracy': [0.8926311135292053, 0.9422486424446106, 0.9523577094078064, 0.95

In [14]:

import PIL
PIL.Image.MAX_IMAGE_PIXELS = 933120000

def convert_tif_to_png(tif_path, png_path):
    os.makedirs(os.path.dirname(png_path), exist_ok=True)
    # Load the TIFF image
    with Image.open(tif_path) as img:
        # Convert to PNG and save
        print(f'Saving image: {os.path.basename(png_path)}')
        img.save(png_path, format='PNG')

pth_images = r'\\10.99.68.52\Kiemendata\Valentina Matos\coda to python\test model\5x\TA'
tif_files = glob(os.path.join(pth_images, '*.tif'))
png_path  = os.path.join(pth_images,'PNG')

for img_path in tif_files:
    png_file = os.path.join(png_path, os.path.basename(img_path[:-4] + ".png"))
    convert_tif_to_png(img_path, png_file)
    

Saving image: SG_013_0061.png
Saving image: SG_014_0016.png


In [7]:
from Semanticseg import semantic_seg
import numpy as np
# Load colormap
pthDL = r'\\10.99.68.52\Kiemendata\Valentina Matos\coda to python\test model\04_19_2024'
with open(os.path.join(pthDL, 'net.pkl'), 'rb') as f:
    data = pickle.load(f)
    model_load = data['model']
    colormap = data['cmap']
    
image_path = r'C:\Users\Valentina\OneDrive - Johns Hopkins\Desktop\test png\testing_io\3.png'  # only PNG IMAGES!!!!!
prediction_mask = semantic_seg(image_path, image_size=1024, model = model_load)

KeyboardInterrupt: 